![Nuclio logo](https://nuclio.school/wp-content/uploads/2018/12/nucleoDS-newBlack.png)

#Transfer Learning con la técnica de Fine Tuning

# Connectar a Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Librerias

In [ ]:
from tensorflow import keras as ks
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import os

from tensorflow.keras.applications import vgg16
from tensorflow.keras.models import Model, load_model

# Variables de entorno

<font color="#FF0000">Aquí definireis vuestra ruta del proyecto de perros y Gatos</font>

In [ ]:
ANCHO_IMAGEN = 150
ALTURA_IMAGEN=150
IMAGE_SIZE = (ANCHO_IMAGEN, ALTURA_IMAGEN)
CANALES_IMAGENES = 3

ruta_archivos = '/content/drive/My Drive/Nuclio_DS_Master/Dogs-vs-cats/'

# Cargaremos datos en NPZ


In [ ]:
dict_npz = np.load(ruta_archivos+'xy_train_img.npz')
x_train_img = dict_npz['x']
y_train_img = dict_npz['y']

dict_npz = np.load(ruta_archivos+'xy_test_img.npz')
x_test_img = dict_npz['x']
y_test_img = dict_npz['y']

dict_npz = np.load(ruta_archivos+'xy_val_img.npz')
x_val_img = dict_npz['x']
y_val_img = dict_npz['y']

In [ ]:
x_train_scaled = x_train_img / 255.
x_test_scaled = x_test_img / 255.
x_val_scaled = x_val_img / 255.

# Montamos la Red Neuronal

## Cargar la parte de extraccion de features de VGG16

In [ ]:
vgg = vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(ANCHO_IMAGEN, ALTURA_IMAGEN, CANALES_IMAGENES))

In [ ]:
vgg.summary()

In [ ]:
output = vgg.layers[-1].output
new_output_layer = ks.layers.Flatten()(output)
vgg_model = Model(vgg.input, new_output_layer)

In [ ]:
vgg_model.summary()

In [ ]:
vgg_model.trainable = True #Abrimos la opción a que se puedan modificar pesos del modelo

for layer in vgg_model.layers:
  layer.trainable = False

In [ ]:
pd.set_option('max_colwidth', None)
layers = [(layer, layer.name, layer.trainable) for layer in vgg_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

In [ ]:
entrenable = False

for layer in vgg_model.layers:
  if layer.name in ['block5_conv1','block4_conv1']:
    entrenable = True
  if entrenable:
    layer.trainable = True
  else: 
    layer.trainable = False

In [ ]:
pd.set_option('max_colwidth', None)
layers = [(layer, layer.name, layer.trainable) for layer in vgg_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

# Montemos la red de VGG16 + Clasificación


In [ ]:
input_shape = vgg_model.output_shape[1]

print('Tamaño de entrada:', input_shape)

In [ ]:
model_with_vgg = ks.Sequential()

model_with_vgg.add(vgg_model)
model_with_vgg.add(ks.layers.Dense(512, activation='relu', input_shape=(input_shape,)))
model_with_vgg.add(ks.layers.Dropout(0.3))
model_with_vgg.add(ks.layers.Dense(512, activation='relu'))
model_with_vgg.add(ks.layers.Dropout(0.3))
model_with_vgg.add(ks.layers.Dense(1, activation='sigmoid'))

model_with_vgg.summary()

# Creamos un optimizador

In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
new_adam = Adam(learning_rate=2e-5)


In [ ]:
model_with_vgg.compile(loss='binary_crossentropy', optimizer=new_adam, metrics=['accuracy'])

# Early stopping!!

In [ ]:
callback = EarlyStopping(monitor='val_loss', patience=5)

# Encoding de etiquetas

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(y_train_img)
y_train_encoded = le.transform(y_train_img)
y_val_encoded = le.transform(y_val_img)
y_test_encoded = le.transform(y_test_img)

In [ ]:
print(np.array(y_train_img))
print(np.array(y_train_encoded))

y_train_encoded

# Entrenar

In [ ]:
history = model_with_vgg.fit(x=x_train_scaled, 
                             y=y_train_encoded, batch_size=32, epochs=20, 
                             callbacks=[callback], 
                             validation_data=(x_val_scaled, y_val_encoded))

# Guardamos el modelo


In [ ]:
model_with_vgg.save(ruta_archivos+"model_with_finetunning_transfer_learning_vgg16.h5")

# Carguemos el modelo con basic Transfer Learning

Aquí recogeremos el modelo y los datos de XY pasados por VGG16 post procesados guardados en NPZ

In [ ]:
model_basic_tl = load_model(ruta_archivos+"model_with_basic_transfer_learning_post_vgg16.h5")

dict_npz = np.load(ruta_archivos+'post_vgg16_test_data.npz')
x_test_post_vgg = dict_npz['x']
y_test_post_vgg = dict_npz['y']


# Miremos que tal rinde el modelo en errores y accuracy

In [ ]:
plt.title('Cross Entropy Loss')
plt.plot(history.history['loss'], color='blue', label='train')
plt.plot(history.history['val_loss'], color='orange', label='test')
plt.show()

plt.title('Classification Accuracy')
plt.plot(history.history['accuracy'], color='blue', label='train')
plt.plot(history.history['val_accuracy'], color='orange', label='test')
plt.show()

In [ ]:
_, acc = model_with_vgg.evaluate(x_test_scaled, y_test_encoded, verbose=0)
print('Modelo con Fine Tunning Transfer Learning > %.3f' % (acc * 100.0))

_, acc = model_basic_tl.evaluate(x_test_post_vgg, y_test_post_vgg, verbose=0)
print('Modelo con Basic Transfer Learning > %.3f' % (acc * 100.0))